# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter
from statistics import mean

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secretsAPI import IEX_CLOUD_API_TOKEN


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.1249947309036134

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
dataframe = pd.DataFrame(columns=my_columns)

for symbols in symbol_strings:
    batchCall = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchCall).json()
    for symbol in symbols.split(','):
      dataframe = dataframe.append(
           pd.Series(
               [
               symbol,
               data[symbol]['price'],
               data[symbol]['stats']['year1ChangePercent'], 
               'NA'
               ], 
               index=my_columns), 
          ignore_index=True
        )

dataframe

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: Futur

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: Futur

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: Futur

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: Futur

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: Futur

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\3428907007.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,AAPL,169.56,0.121908,NA
1,MSFT,279.38,-0.015177,NA
2,AMZN,140.02,-0.195019,NA
3,TSLA,896.70,0.299687,NA
4,BRK.B,298.90,0.082624,NA
...,...,...,...,...
498,PENN,35.53,-0.504013,NA
499,VNO,30.20,-0.274803,NA
500,ALK,45.78,-0.243051,NA
501,PVH,63.59,-0.421405,NA


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
dataframe = dataframe[:50]
dataframe.reset_index(inplace=True)
dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,189,DVN,63.950,1.615537,NA
1,133,OXY,68.150,1.566681,NA
2,366,MRO,24.230,1.223140,NA
3,327,CF,96.190,1.093330,NA
4,439,APA,36.429,1.050523,NA
5,284,CTRA,29.830,0.932276,NA
6,60,COP,96.600,0.823880,NA
7,10,XOM,97.620,0.781860,NA
8,138,PXD,232.600,0.780477,NA
9,161,VLO,114.520,0.747517,NA


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def getPortfolioSize():
    invalid_portfolio_size = True
    while (invalid_portfolio_size):
        portfolio_size = input("Enter Portfolio Value: ")
        try:
            val = float(portfolio_size)
            invalid_portfolio_size = False
        except ValueError:
            print("Input must be a valid number")
    return val

Enter Portfolio Value: 1000000


In [9]:
portfolioSize = portfolioSize / len(dataframe)

for i in range(len(dataframe)):
    dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(portfolioSize / dataframe.loc[i, 'Price'])
    
dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,189,DVN,63.950,1.615537,312
1,133,OXY,68.150,1.566681,293
2,366,MRO,24.230,1.223140,825
3,327,CF,96.190,1.093330,207
4,439,APA,36.429,1.050523,549
5,284,CTRA,29.830,0.932276,670
6,60,COP,96.600,0.823880,207
7,10,XOM,97.620,0.781860,204
8,138,PXD,232.600,0.780477,85
9,161,VLO,114.520,0.747517,174


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqmCol = [
    'Ticker',
    'Price',
    'Number of Shares To Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

dataframeHQM = pd.DataFrame(columns = hqmCol)

for symbols in symbol_strings:
    batchCall = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchCall).json()
    for symbol in symbols.split(','):
        dataframeHQM = dataframeHQM.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]["stats"]['year1ChangePercent'],
                    'N/A',
                    data[symbol]["stats"]['month6ChangePercent'],
                    'N/A',
                    data[symbol]["stats"]['month3ChangePercent'],
                    'N/A',
                    data[symbol]["stats"]['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqmCol),
                ignore_index = True
        )
                             
dataframeHQM

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_

C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(
C:\Users\kr1sh\AppData\Local\Temp\ipykernel_18204\4050600185.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframeHQM = dataframeHQM.append(


,Ticker,Price,Number of Shares To Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,165.06,N/A,0.121423,N/A,-0.070437,N/A,0.032590,N/A,0.193996,N/A,N/A
1,MSFT,282.41,N/A,-0.015065,N/A,-0.099186,N/A,0.012134,N/A,0.095349,N/A,N/A
2,AMZN,138.04,N/A,-0.191448,N/A,-0.102236,N/A,0.089974,N/A,0.280565,N/A,N/A
3,TSLA,935.49,N/A,0.298380,N/A,-0.049168,N/A,0.024531,N/A,0.328281,N/A,N/A
4,BRK.B,308.06,N/A,0.080721,N/A,-0.039958,N/A,-0.069461,N/A,0.103118,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,PENN,34.95,N/A,-0.518004,N/A,-0.251680,N/A,-0.056227,N/A,0.142409,N/A,N/A
499,VNO,31.20,N/A,-0.273705,N/A,-0.251005,N/A,-0.211555,N/A,0.066083,N/A,N/A
500,ALK,45.39,N/A,-0.245703,N/A,-0.197956,N/A,-0.186767,N/A,0.111652,N/A,N/A
501,PVH,63.48,N/A,-0.423156,N/A,-0.348147,N/A,-0.154237,N/A,0.091307,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

dataframeHQM.dropna(inplace=True)
for row in dataframeHQM.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        dataframeHQM.loc[row, percentile_col] = score(dataframeHQM[change_col],
                                                           dataframeHQM.loc[row, change_col]) / 100
dataframeHQM

,Ticker,Price,Number of Shares To Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,165.06,N/A,0.121423,75.745527,-0.070437,49.502982,0.032590,64.015905,0.193996,93.04175,N/A
1,MSFT,282.41,N/A,-0.015065,56.461233,-0.099186,42.544732,0.012134,55.864811,0.095349,52.087475,N/A
2,AMZN,138.04,N/A,-0.191448,24.45328,-0.102236,41.351889,0.089974,82.703777,0.280565,97.614314,N/A
3,TSLA,935.49,N/A,0.298380,91.650099,-0.049168,55.26839,0.024531,61.033797,0.328281,98.60835,N/A
4,BRK.B,308.06,N/A,0.080721,70.178926,-0.039958,57.455268,-0.069461,25.646123,0.103118,57.256461,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,PENN,34.95,N/A,-0.518004,2.385686,-0.251680,9.145129,-0.056227,28.230616,0.142409,77.733598,N/A
499,VNO,31.20,N/A,-0.273705,14.910537,-0.251005,9.542744,-0.211555,3.976143,0.066083,39.16501,N/A
500,ALK,45.39,N/A,-0.245703,17.693837,-0.197956,17.693837,-0.186767,6.163022,0.111652,62.823062,N/A
501,PVH,63.48,N/A,-0.423156,4.970179,-0.348147,3.379722,-0.154237,9.145129,0.091307,50.298211,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
for row in dataframeHQM.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(dataframeHQM.loc[row, f'{time_period} Return Percentile'])
    dataframeHQM.loc[row, 'HQM Score'] = mean(momentum_percentiles)
dataframeHQM

,Ticker,Price,Number of Shares To Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,165.06,N/A,0.121423,75.745527,-0.070437,49.502982,0.032590,64.015905,0.193996,93.04175,70.576541
1,MSFT,282.41,N/A,-0.015065,56.461233,-0.099186,42.544732,0.012134,55.864811,0.095349,52.087475,51.739563
2,AMZN,138.04,N/A,-0.191448,24.45328,-0.102236,41.351889,0.089974,82.703777,0.280565,97.614314,61.530815
3,TSLA,935.49,N/A,0.298380,91.650099,-0.049168,55.26839,0.024531,61.033797,0.328281,98.60835,76.640159
4,BRK.B,308.06,N/A,0.080721,70.178926,-0.039958,57.455268,-0.069461,25.646123,0.103118,57.256461,52.634195
...,...,...,...,...,...,...,...,...,...,...,...,...
498,PENN,34.95,N/A,-0.518004,2.385686,-0.251680,9.145129,-0.056227,28.230616,0.142409,77.733598,29.373757
499,VNO,31.20,N/A,-0.273705,14.910537,-0.251005,9.542744,-0.211555,3.976143,0.066083,39.16501,16.898608
500,ALK,45.39,N/A,-0.245703,17.693837,-0.197956,17.693837,-0.186767,6.163022,0.111652,62.823062,26.093439
501,PVH,63.48,N/A,-0.423156,4.970179,-0.348147,3.379722,-0.154237,9.145129,0.091307,50.298211,16.94831


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
dataframeHQM.sort_values('HQM Score', ascending=False, inplace=True)
dataframeHQM = dataframeHQM[:50]
dataframeHQM.reset_index(inplace=True, drop=True)
dataframeHQM

,Ticker,Price,Number of Shares To Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ENPH,288.56,N/A,0.508284,96.222664,1.050626,100.0,0.773004,100.0,0.459376,100.0,99.055666
1,SEDG,362.68,N/A,0.388711,94.632207,0.532513,99.602386,0.459896,99.801193,0.331653,99.403579,98.359841
2,ON,65.47,N/A,0.712539,97.415507,0.133578,88.469185,0.282797,99.204771,0.343182,99.602386,96.172962
3,CDNS,185.33,N/A,0.264096,89.860835,0.233088,94.831014,0.239191,98.60835,0.247151,97.017893,95.079523
4,SNPS,377.24,N/A,0.282257,91.053678,0.184119,92.445328,0.290631,99.403579,0.214009,95.228628,94.532803
5,CTRA,29.98,N/A,0.942847,99.005964,0.463331,99.403579,0.082509,81.11332,0.188918,91.650099,92.793241
6,OXY,66.41,N/A,1.563800,99.801193,0.774322,99.801193,0.198502,96.819085,0.118315,66.40159,90.705765
7,XOM,96.25,N/A,0.803114,98.807157,0.303498,97.2167,0.148738,92.842942,0.132368,73.161034,90.506958
8,DVN,63.02,N/A,1.675681,100.0,0.292889,97.017893,0.102161,85.884692,0.141872,77.335984,90.059642
9,CEG,65.45,N/A,0.254457,88.866799,0.264987,96.023857,0.124341,90.258449,0.161187,83.101392,89.562624


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
position_size = float(portfolioSize/len(dataframeHQM.index))

In [15]:
for i in dataframeHQM.index:
    dataframeHQM.loc[i, 'Number of Shares To Buy'] = math.floor(position_size / dataframeHQM.loc[i, 'Price'])
dataframeHQM

,Ticker,Price,Number of Shares To Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ENPH,288.56,1,0.508284,96.222664,1.050626,100.0,0.773004,100.0,0.459376,100.0,99.055666
1,SEDG,362.68,1,0.388711,94.632207,0.532513,99.602386,0.459896,99.801193,0.331653,99.403579,98.359841
2,ON,65.47,6,0.712539,97.415507,0.133578,88.469185,0.282797,99.204771,0.343182,99.602386,96.172962
3,CDNS,185.33,2,0.264096,89.860835,0.233088,94.831014,0.239191,98.60835,0.247151,97.017893,95.079523
4,SNPS,377.24,1,0.282257,91.053678,0.184119,92.445328,0.290631,99.403579,0.214009,95.228628,94.532803
5,CTRA,29.98,13,0.942847,99.005964,0.463331,99.403579,0.082509,81.11332,0.188918,91.650099,92.793241
6,OXY,66.41,6,1.563800,99.801193,0.774322,99.801193,0.198502,96.819085,0.118315,66.40159,90.705765
7,XOM,96.25,4,0.803114,98.807157,0.303498,97.2167,0.148738,92.842942,0.132368,73.161034,90.506958
8,DVN,63.02,6,1.675681,100.0,0.292889,97.017893,0.102161,85.884692,0.141872,77.335984,90.059642
9,CEG,65.45,6,0.254457,88.866799,0.264987,96.023857,0.124341,90.258449,0.161187,83.101392,89.562624


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
dataframeHQM.to_excel(writer, 'Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [19]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_format = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [20]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['One-Year Price Return', percent_format],
    'E': ['One-Year Return Percentile', percent_format],
    'F': ['Six-Month Price Return', percent_format],
    'G': ['Six-Month Return Percentile', percent_format],
    'H': ['Three-Month Price Return', percent_format],
    'I': ['Three-Month Return Percentile', percent_format],
    'J': ['One-Month Price Return', percent_format],
    'K': ['One-Month Return Percentile', percent_format],
    'L': ['HQM Score', integer_format]
    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [21]:
writer.save()